In [2]:
import csv
import json
import re

def parse_error(error_str):
    """Extract the core error message from the traceback"""
    lines = error_str.strip().split('\n')
    return lines[-1].strip() if lines else "No error message found"

def parse_kwargs(kwargs_str):
    """Parse kwargs string into a dictionary and extract key scheduling info"""
    # Replace single quotes with double quotes and None with null
    kwargs_str = kwargs_str.replace("'", '"').replace("None", "null")
    # Handle datetime and timedelta more robustly
    kwargs_str = re.sub(r'datetime\.datetime\((.*?)\)', r'"\1"', kwargs_str)
    kwargs_str = re.sub(r'datetime\.timedelta\((.*?)\)', r'"\1"', kwargs_str)
    
    try:
        kwargs = json.loads(kwargs_str)
    except json.JSONDecodeError as e:
        return {
            "raw": kwargs_str,
            "error": f"Failed to parse kwargs: {str(e)}"
        }
    
    # Extract key scheduling constraints into a structured format
    summary = {
        "asset_id": kwargs.get("asset_or_sensor", {}).get("id"),
        "start": kwargs.get("start"),
        "end": kwargs.get("end"),
        "resolution": kwargs.get("resolution"),
        "soc_at_start": kwargs.get("flex_model", {}).get("soc-at-start"),
        "soc_min": kwargs.get("flex_model", {}).get("soc-min"),
        "soc_max": kwargs.get("flex_model", {}).get("soc-max"),
        "soc_unit": kwargs.get("flex_model", {}).get("soc-unit"),
    }
    
    if "soc-minima" in kwargs.get("flex_model", {}):
        summary["soc_minima"] = kwargs["flex_model"]["soc-minima"]
    if "soc-maxima" in kwargs.get("flex_model", {}):
        summary["soc_maxima"] = kwargs["flex_model"]["soc-maxima"]
    if "consumption-capacity" in kwargs.get("flex_model", {}):
        summary["consumption_capacity"] = kwargs["flex_model"]["consumption-capacity"]
    if "production-capacity" in kwargs.get("flex_model", {}):
        summary["production_capacity"] = kwargs["flex_model"]["production-capacity"]
    
    return summary

def process_failed_jobs(csv_file):
    structured_data = []
    
    try:
        with open(csv_file, 'r', newline='') as f:
            reader = csv.reader(f)
            next(reader)  # Skip header
            
            for row_num, row in enumerate(reader, start=2):
                if len(row) != 6:
                    print(f"Skipping row {row_num}: Expected 6 columns, got {len(row)} - {row}")
                    continue
                
                job_id, error, kwargs, func_name, started_at, ended_at = row
                
                structured_job = {
                    "Job ID": job_id,
                    "Error": parse_error(error),
                    "Kwargs": parse_kwargs(kwargs),
                    "Function Name": func_name,
                    "Started At": started_at,
                    "Ended At": ended_at
                }
                structured_data.append(structured_job)
    except FileNotFoundError:
        print(f"Error: '{csv_file}' not found.")
        return []
    except Exception as e:
        print(f"Error reading CSV: {e}")
        return []
    
    return structured_data

def print_dict_with_newlines(data, indent=2):
    """Helper function to print dictionary with each key-value pair on a new line"""
    if isinstance(data, dict):
        for key, value in data.items():
            if isinstance(value, (dict, list)):
                print(f"{' ' * indent}{key}:")
                print_dict_with_newlines(value, indent + 2)
            else:
                print(f"{' ' * indent}{key}: {value}")
    elif isinstance(data, list):
        for i, item in enumerate(data):
            print(f"{' ' * indent}Item {i}:")
            print_dict_with_newlines(item, indent + 2)
    else:
        print(f"{' ' * indent}{data}")

def print_all_jobs_with_index(jobs):
    if not jobs:
        print("No jobs to display.")
        return
    
    for index, job in enumerate(jobs):
        print(f"\nJob Index: {index}")
        print(f"Job ID: {job['Job ID']}")
        print(f"Error: {job['Error']}")
        print(f"Function Name: {job['Function Name']}")
        print(f"Started At: {job['Started At']}")
        print(f"Ended At: {job['Ended At']}")
        print("Scheduling Constraints:")
        print_dict_with_newlines(job['Kwargs'])
        print("-" * 50)

if __name__ == "__main__":
    csv_file = 'failed_jobs.csv'
    jobs = process_failed_jobs(csv_file)
    print_all_jobs_with_index(jobs)


Job Index: 0
Job ID: c08f782c-9dfb-4d62-90be-f359ed2a6c7a
Error: flexmeasures.data.models.planning.exceptions.InfeasibleProblemException
Function Name: flexmeasures.data.services.scheduling.make_schedule
Started At: 2025-02-20 13:51:46.038200
Ended At: 2025-02-20 13:51:47.062469
Scheduling Constraints:
  raw: {"asset_or_sensor": {"id": 262, "class": "Sensor"}, "scheduler_specs": null, "start": "2025, 2, 20, 15, 0, tzinfo=datetime.timezone("seconds=3600", "+0100""), "end": "2025, 2, 21, 15, 0, tzinfo=datetime.timezone("seconds=3600", "+0100""), "resolution": "seconds=900", "belief_time": null, "flex_model": {"soc-unit": "kWh", "soc-at-start": 8.0001, "soc-max": 65.001, "soc-min": 24.001}, "flex_context": {"consumption-price-sensor": 14, "production-price-sensor": 14}}
  error: Failed to parse kwargs: Expecting ',' delimiter: line 1 column 134 (char 133)
--------------------------------------------------

Job Index: 1
Job ID: c5140205-829b-44be-9ff9-b863f5fb4a30
Error: flexmeasures.data